# SENTIMENT ANALYSIS

In [3]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score

import json

### Load Data

#### Data Class

In [4]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: # Score of 4 or 5
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

#### Loading data

In [5]:
file_name = '/Users/nitishsolanki/Python/Python/Projects/DS Projects/Scikit Learn/data/Books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

### Prep Data

In [6]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [7]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


#### Bag of words vectorization

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# This book is great!
# This book is so bad

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)
print(train_x[0])
print(train_x_vectors[0].toarray())


It is not worth 1 cent!You should remove it from a list of books anyone should actually pay for.If I wrote something like this in the third grade I would expect to get an &#34;F&#34;It was a total let down!It was written by a Spitfire hater ... justifying the Hurricane in an annoying and sulky way!
[[0. 0. 0. ... 0. 0. 0.]]


### Classification  

#### Linear SVM

In [9]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear', C=1)

clf_svm.fit(train_x_vectors, train_y)

clf_svm.predict(test_x_vectors[0])



array(['POSITIVE'], dtype='<U8')

#### Decision Tree

In [10]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

#### Naive Bayes

In [11]:
from sklearn.naive_bayes import GaussianNB

train_x_vectors_dense = train_x_vectors.toarray()

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors_dense, train_y)

test_x_vectors_dense = test_x_vectors.toarray()
test_x_2d = test_x_vectors_dense[0].reshape(1, -1)

clf_gnb.predict(test_x_2d)

array(['NEGATIVE'], dtype='<U8')

#### Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Evaluation

In [13]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
# print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6177884615384616
0.8052884615384616


In [14]:
# F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [15]:
test_set = ['I thoroughly enjoyed this, 5 stars', "bad book do not buy", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

### Tuning our model (with Grid Search)

In [16]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)}

svc = SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)
print(clf.best_estimator_)

SVC(C=4)


In [17]:
print(clf.score(test_x_vectors, test_y))

0.8197115384615384


### Saving Model

##### Save model

In [18]:
import pickle

with open('/Users/nitishsolanki/Python/Python/Projects/DS Projects/Scikit Learn/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)
f.close()

##### Load model

In [19]:
with open('/Users/nitishsolanki/Python/Python/Projects/DS Projects/Scikit Learn/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [20]:
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

What a wonderful story this is. Didn't want to put the book down. How sad what children were put though.


array(['POSITIVE'], dtype='<U8')